# Web Scraping Instagram with Selenium 

### Paso 0: Importar librerías

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait

import time

import requests
from bs4 import BeautifulSoup

# Para eliminar elementos duplicados en lista sin afectar el orden
from collections import OrderedDict

# Descargar y guardar las imágenes
import os
import wget

### Paso 1: Descargar ChromeDriver
Se necesita descargar la última versión estable de ChromeDriver:
<br>
https://chromedriver.chromium.org/
<br>

### Paso 2: Iniciar sesión en tu cuenta de Instagram

In [2]:
# Manipular propiedades driver de Chrome
chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument("start-maximized") # Maximizar la pestaña de Chrome
chrome_options.add_argument("--disable-extensions") # Deshabilitar extensiones


In [3]:
# Añadir credenciales de la cuenta
username_ = '' # tu username o correo
password_ = '' # tu password

In [4]:
# Especificar la ruta a chromedriver.exe (en este caso, el notebook está en la misma ruta con el chromedriver.exe)
driver = webdriver.Chrome('chromedriver.exe', options =chrome_options)

# Abrir la página web
driver.get("http://www.instagram.com")

# Ubicar campos username y password en la página web de Instagram
username = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='username']")))
password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='password']")))

# Ingresar username y password
username.clear()
username.send_keys(username_)
password.clear()
password.send_keys(password_)

# Ubicar al botón de 'iniciar sesión' y hacer clic en él
button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()

### Paso 3: Evitar alertas
Es posible que solo se reciba una o dos alertas. Dependiendo el caso, ajustar la celda siguiente.

In [5]:
# Esperar 2 segundos para que cargue la página
time.sleep(2) 

# Evitar alerta(s)
alert = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "Ahora no")]'))).click() # Guardar información de sesión - No ahora
alert2 = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "Ahora no")]'))).click() # Aparece un pop up de "Activar notificaciones"
# alert = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "Not Now")]'))).click() # Si es en inglés
# alert2 = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "Not Now")]'))).click() # Si es en inglés

### Paso 4: Buscar hashtag (keyword) deseado

In [6]:
# Ubicar el campo "buscar"
searchbox = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//input[@placeholder='Buscar']"))) # Si es en español
# searchbox = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//input[@placeholder='Search']"))) # Si es en inglés
searchbox.clear()

# Buscar el hashtag
keyword = "#clairo"
searchbox.send_keys(keyword)
 
# Asegurarse de que busque el hashtag (a veces requiere que se clickee dos veces para que pueda realizar la búsqueda)
time.sleep(5) # Esperar 5 segundos para que cargue la página
link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[contains(@href, '/" + keyword[1:] + "/')]")))
link.click()

### Paso 5: Obtener links de posts que se encontraron buscando el hashtag
Aumente n_scrolls para seleccionar más posts (según la resolución de la pantalla)
<br>
<b>Ejemplo:</b>
<br>
<ul>
    <li>2 scrolls -> ~ 35 photos</li>
    <li>3 scrolls -> ~ 45 photos</li>
</ul>

In [7]:
# Aumentar el rango para scrollear más
n_scrolls = 10
for j in range(0, n_scrolls):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") # Scrollear hasta donde la página cargue
    time.sleep(5)

In [8]:
# Ubicar a todos los elementos de enlace en la página
hashtag_posts = driver.find_elements_by_tag_name('a')
hashtag_posts = [a.get_attribute('href') for a in hashtag_posts]

# Limitar todos los enlaces a enlaces netamente de imágenes
hashtag_posts = [a for a in hashtag_posts if str(a).startswith("https://www.instagram.com/p/")]

print('Se encontraron ' + str(len(hashtag_posts)) + ' links de posts (con el hashtag)')
hashtag_posts[:5]

Se encontraron 54 links de posts (con el hashtag)


['https://www.instagram.com/p/CT47lxIKFhe/',
 'https://www.instagram.com/p/CSunjvmr2lI/',
 'https://www.instagram.com/p/CUU8aq6Pp5H/',
 'https://www.instagram.com/p/CUlDUjhvT5I/',
 'https://www.instagram.com/p/CTU_Zcoq8GH/']

### Paso 6: De posts (que tienen el hashtag) a los perfiles (que los postearon)

In [9]:
perfiles = [] # Lista que almacena los perfiles de Instagram

for i in hashtag_posts:
    driver.get(i)
    soup = BeautifulSoup(driver.page_source,"html.parser")
    div = soup.find_all('div', {'class':'e1e1d'}) # clase 'div' que comprende el id del perfil
    for c in div:
        a = c.find_all('a') # clase 'a' que comprende el id del perfil
        for e in a:
            perfil = "https://www.instagram.com" + e['href'] # 'href' comprende el id del perfil
            perfiles.append(perfil) # Añadir link de perfil a la lista
            
perfiles = list(OrderedDict.fromkeys(perfiles))
perfiles

['https://www.instagram.com/arttdump/',
 'https://www.instagram.com/yandhi_memes/',
 'https://www.instagram.com/clairo___tw/',
 'https://www.instagram.com/clairodailypics/',
 'https://www.instagram.com/wilmonandevakslovebot/',
 'https://www.instagram.com/clairoshawtybae/',
 'https://www.instagram.com/sinkinclairo/',
 'https://www.instagram.com/clairozinnias/',
 'https://www.instagram.com/clairo_supremacy/',
 'https://www.instagram.com/nervouscouldnttellyouwhy/',
 'https://www.instagram.com/_clairo_love_/',
 'https://www.instagram.com/alewife.fan/',
 'https://www.instagram.com/clairowine/',
 'https://www.instagram.com/clairosimp_/',
 'https://www.instagram.com/thetribaltattoostudioranchi/',
 'https://www.instagram.com/clairoreport/',
 'https://www.instagram.com/clairobestpics/',
 'https://www.instagram.com/clairogoddess/',
 'https://www.instagram.com/bjozone/',
 'https://www.instagram.com/kiarais.bae/',
 'https://www.instagram.com/clairoiscute/',
 'https://www.instagram.com/pre.ttyclair

### Paso 7: De perfiles (con imgs del hashtag) a links de sus posts

In [10]:
# Perfiles que no postean frecuentemente sobre el hashtag o tiene baja calidad de imágenes
no_perfiles = {'https://www.instagram.com/arttdump/', 'https://www.instagram.com/yandhi_memes/', 'https://www.instagram.com/wilmonandevakslovebot/',
               'https://www.instagram.com/thetribaltattoostudioranchi/', 'https://www.instagram.com/kiarais.bae/', 'https://www.instagram.com/heartsce/',
               'https://www.instagram.com/junichi_kobe/', 'https://www.instagram.com/luv.clair0/'}
               
# Quitamos los perfiles no deseados
perfiles = [i for i in perfiles if i not in no_perfiles] 
perfiles

['https://www.instagram.com/clairo___tw/',
 'https://www.instagram.com/clairodailypics/',
 'https://www.instagram.com/clairoshawtybae/',
 'https://www.instagram.com/sinkinclairo/',
 'https://www.instagram.com/clairozinnias/',
 'https://www.instagram.com/clairo_supremacy/',
 'https://www.instagram.com/nervouscouldnttellyouwhy/',
 'https://www.instagram.com/_clairo_love_/',
 'https://www.instagram.com/alewife.fan/',
 'https://www.instagram.com/clairowine/',
 'https://www.instagram.com/clairosimp_/',
 'https://www.instagram.com/clairoreport/',
 'https://www.instagram.com/clairobestpics/',
 'https://www.instagram.com/clairogoddess/',
 'https://www.instagram.com/bjozone/',
 'https://www.instagram.com/clairoiscute/',
 'https://www.instagram.com/pre.ttyclairo/',
 'https://www.instagram.com/clairecottrillisperfect/',
 'https://www.instagram.com/clairosflaminghotcheetos/',
 'https://www.instagram.com/clairosfavoriteblouse/']

In [11]:
# Lista para almacenar todos los links de los posts
todos_posts = []

for perfil in perfiles:
    driver.get(perfil) # Abrir la página del perfil

    # Definir número de scrolls para cada perfil (scrollee eficientemente)
    soup = BeautifulSoup(driver.page_source,"html.parser")
    n_publis = soup.find_all('span', {'class':'g47SY'})
    n_publis = int(n_publis[0].text.replace(',', '')) # Eliminar las comas de # de "publicaciones" para convertir a integer
    n_scrolls = round(n_publis/12) # Calcular número de scrolls
    # n_scrolls = 4 # 1 = 24, 2 = 36, 3 =37(42), 4=
    
    # Scrollear el perfil
    for j in range(0, n_scrolls):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") # Scrollear hasta donde la página cargue
        time.sleep(4)
    
    # Obtener links de las fotos
    fotos = driver.find_elements_by_tag_name('a')
    fotos = [a.get_attribute('href') for a in fotos]
    
    # Consolidar todos los links de los posts
    todos_posts = fotos + todos_posts
    
# Limitar todos los enlaces a enlaces netamente de imágenes
todos_posts = [a for a in todos_posts if str(a).startswith("https://www.instagram.com/p/")]

print('Se encontraron ' + str(len(todos_posts)) + ' links de posts')
todos_posts[:5]

Se encontraron 240 links de posts


['https://www.instagram.com/p/CUH87JzLiQQ/',
 'https://www.instagram.com/p/CUAVRi8LRkL/',
 'https://www.instagram.com/p/CTuU6DWLLjQ/',
 'https://www.instagram.com/p/CSuagRILfxI/',
 'https://www.instagram.com/p/CRnQBSDroWl/']

### Paso 8: De links de posts a links de las imágenes (de esos posts)

In [12]:
def ObtenerImagen(driver):
    img = driver.find_elements_by_class_name('KL4Bh')[-1].find_element_by_tag_name('img').get_attribute('src')
    return img

In [17]:
img_urls = [] # Lista que almacena los urls de las fotos (imágenes) de los posts

for post in todos_posts:
    count = 0 # Contador de la cantidad de errores
    driver.get(post)
    verificar_imagen = driver.find_elements_by_class_name('KL4Bh') # La clase 'KL4Bh' es para solo imágenes
    if verificar_imagen:
        img_1 = driver.find_elements_by_class_name('KL4Bh')[0].find_element_by_tag_name('img').get_attribute('src')
        img_urls.append(img_1) # Agregar la imagen inicial (si el post solo tiene una foto o comprende una serie de fotos)
        img_urls.append(ObtenerImagen(driver)) # Agregar la última imagen de la serie de fotos del post antes del deslizamiento
        while True:
            try:
                elements = driver.find_elements_by_class_name('_6CZji') # la clase '_6CZji' es el ícono de deslizar a la derecha
                elements[0].click() # Clickear en el ícono de deslizar a la derecha (las fotos en la serie de fotos del post)
                img_urls.append(ObtenerImagen(driver)) # Agregar la última imagen hasta el momento del último deslizamiento 
                                                       # donde ha cargado las fotos (en la serie de fotos del post)
            except:
                count+=1
                time.sleep(1)
                if count == 2:
                    break 
img_urls = list(OrderedDict.fromkeys(img_urls)) # Eliminar duplicidad manteniendo el orden
img_urls = [i for i in img_urls if i] # Eliminar valores None
img_urls[:5]

['https://scontent-lim1-1.cdninstagram.com/v/t51.2885-15/e35/p1080x1080/242728364_813554632650932_4593528210585341559_n.jpg?_nc_ht=scontent-lim1-1.cdninstagram.com&_nc_cat=108&_nc_ohc=Sxkt9OrAlrsAX-mmevO&edm=AABBvjUBAAAA&ccb=7-4&oh=ede88c7566b40388328675823aeb1db8&oe=616202EE&_nc_sid=83d603',
 'https://scontent-lim1-1.cdninstagram.com/v/t51.2885-15/e35/p1080x1080/242286173_1243805482732593_8111869125281077374_n.jpg?_nc_ht=scontent-lim1-1.cdninstagram.com&_nc_cat=109&_nc_ohc=qwnOr1939FUAX9WlRAt&edm=AABBvjUBAAAA&ccb=7-4&oh=ad5802211e9c04bac1c818891494b9f1&oe=61608FB1&_nc_sid=83d603',
 'https://scontent-lim1-1.cdninstagram.com/v/t51.2885-15/e35/241731045_769855143769203_7944280748832899072_n.jpg?_nc_ht=scontent-lim1-1.cdninstagram.com&_nc_cat=103&_nc_ohc=zc8pCLDiXGcAX-v97tN&edm=AABBvjUBAAAA&ccb=7-4&oh=b2bed3a9e1553efba2b222b1ea9b001d&oe=6160AA57&_nc_sid=83d603',
 'https://scontent-lim1-1.cdninstagram.com/v/t51.2885-15/e35/p1080x1080/239186197_168705652009420_2500254429837105553_n.jpg?_nc_

### Paso 9: Descargar y guardar imágenes en la computadora

Primero se crea una nueva carpeta para las imágenes en cierto directorio. Luego, se guardan todas las imágenes allí.

In [18]:
import os
import wget

path = os.getcwd()
path = os.path.join(path, keyword[1:] + "_dataset")

#create the directory
os.mkdir(path)

path

'C:\\Users\\SANDRO\\Projects\\Clairo Image Recognition\\clairo_dataset'

In [19]:
#download images
counter = 0
for image in img_urls:
    save_as = os.path.join(path, keyword[1:] + str(counter) + '.jpg')
    print(save_as)
    wget.download(image, save_as)
    counter += 1

C:\Users\SANDRO\Projects\Clairo Image Recognition\clairo_dataset\clairo0.jpg
100% [............................................................................] 174945 / 174945C:\Users\SANDRO\Projects\Clairo Image Recognition\clairo_dataset\clairo1.jpg
100% [............................................................................] 212938 / 212938C:\Users\SANDRO\Projects\Clairo Image Recognition\clairo_dataset\clairo2.jpg
100% [..............................................................................] 89199 / 89199C:\Users\SANDRO\Projects\Clairo Image Recognition\clairo_dataset\clairo3.jpg
100% [............................................................................] 193246 / 193246C:\Users\SANDRO\Projects\Clairo Image Recognition\clairo_dataset\clairo4.jpg
100% [..............................................................................] 31807 / 31807C:\Users\SANDRO\Projects\Clairo Image Recognition\clairo_dataset\clairo5.jpg
100% [.....................................